<a href="https://colab.research.google.com/github/Binamra00/paint-paint/blob/main/notebooks/domain_inpaint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paint-Paint: Domain Adaptive Image Inpainting
**Phase 3: Environment Setup & Bootstrapping**

This notebook is self-contained. It pulls the latest version of the `paint-paint` codebase from GitHub into the Colab environment, sets up the Python path for modular importing, and installs all strict dependencies.

In [ ]:
import os
import sys

# 0. FORCE Colab to anchor to the default root directory
%cd /content

# 1. Clone the repository into Colab's local runtime
REPO_DIR = '/content/paint-paint'
if not os.path.exists(REPO_DIR):
    print("Cloning the codebase from GitHub...")
    !git clone https://github.com/Binamra00/paint-paint.git
else:
    print("Repository already exists. Pulling latest updates...")
    %cd "{REPO_DIR}"
    !git pull

# 2. Change working directory to the repo and add it to the Python path
%cd "{REPO_DIR}"
if REPO_DIR not in sys.path:
    sys.path.append(REPO_DIR)

# 3. Install strict dependencies
!pip install -q -r requirements.txt
print("Environment Ready!")

/content
Cloning the codebase from GitHub...
Cloning into 'paint-paint'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 80 (delta 33), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 32.41 KiB | 16.21 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/paint-paint
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 18.7 MB/s eta 0:00:00
Environment Ready!


## Pipeline Stage 1: Data Ingestion & Reproducibility
To ensure experiments can be globally reproduced, all random number generators are strictly locked. We then execute our custom `download_oxford.py` module to fetch the dataset.

In [ ]:
import torch
import numpy as np
import random

# Lock seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True

# Execute the custom data ingestion module
!python data/download_oxford.py

2026-02-28 19:56:09,541 - INFO - Downloading Oxford 102 Flowers from https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz...
2026-02-28 19:56:36,046 - INFO - Download complete!
2026-02-28 19:56:36,046 - INFO - Extracting /content/paint-paint/data/raw/102flowers.tgz to /content/paint-paint/data/raw/oxford_102...
/content/paint-paint/data/download_oxford.py:43: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=EXTRACT_DIR)
2026-02-28 19:56:38,846 - INFO - Extraction complete! Dataset is ready for the pipeline.


## Pipeline Stage 2: Model Architecture Initialization
To conclude Phase 3, we initialize our two core models.
1. **The Baseline:** A custom U-Net utilizing Gated Convolutions to mitigate color-bleeding from masked regions.
2. **The Target:** We stage the environment to load the BigLaMa architecture, which utilizes Fast Fourier Convolutions (FFCs) for a global receptive field.

In [ ]:
from models.unet_gated import BaselineUNet
from models.bigLaMa import fetch_pretrained_lama

# 1. Instantiate the Baseline Model
print("Initializing Baseline U-Net with Gated Convolutions...")
baseline_model = BaselineUNet()

# Calculate parameters to prove the model is fully loaded in memory
total_params = sum(p.numel() for p in baseline_model.parameters() if p.requires_grad)
print(f"-> Baseline U-Net successfully loaded with {total_params:,} learnable parameters.\n")

# 2. Stage the BigLaMa Transfer Learning Weights
print("Staging BigLaMa Architecture...")
fetch_pretrained_lama()

print("\n✅ Phase 3 Complete: Environment, Data, and Models are fully prepared for Phase 4 experiments.")

Initializing Baseline U-Net with Gated Convolutions...
-> Baseline U-Net successfully loaded with 3,317,891 learnable parameters.

Staging BigLaMa Architecture...

✅ Phase 3 Complete: Environment, Data, and Models are fully prepared for Phase 4 experiments.
